# Kerékpárbérlés előrejelzése – portfólió notebook (napi és órás adatok)

Ez a notebook az első komolyabb gépi tanulás projektem összefoglalója. Célom: a kerékpár‑bérlések számának előrejelzése időjárási és naptári (szezonális) jellemzők alapján.
A tartalom végigvezeti a teljes folyamatot: adatmegismerés → EDA → feature engineering → baseline modell → SHAP értelmezés → finomított (refined) pipeline → értékelés.


## Project Overview

Ebben a projektben a kerékpárbérlések számát (célváltozó: `cnt`) szeretném előrejelezni napi és órás bontásban.
Az adatok a Bike Sharing Dataset-ből származnak (Kaggle). A munkát úgy szerveztem, hogy bármikor újrafuttatható legyen: minden lépés reprodukálható és tisztán dokumentált.


## Importok és alapbeállítás

Itt töltöm be a szükséges csomagokat. Ha valami hiányzik, először ezt a cellát nézd át.


In [ ]:
# Könyvtárak
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, TimeSeriesSplit, cross_val_score, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.inspection import permutation_importance

sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = (8, 4)


## Adatbetöltés

A saját notebookomban ezt a részt már megírtam, de ide is beteszek egy működő változatot.
Ha máshonnan olvasod be az adatot, cseréld ki az elérési útvonalakat.


In [ ]:
# Fájlok elérési útvonala (ha máshol vannak, állítsd át)
DAY_PATH = '/mnt/data/day.csv'   # napi adatok
HOUR_PATH = '/mnt/data/hour.csv' # órás adatok

# Betöltés
df_daily = pd.read_csv(DAY_PATH)
df_hour  = pd.read_csv(HOUR_PATH)

# Típusok, dátum konverzió
if 'dteday' in df_daily.columns:
    df_daily['dteday'] = pd.to_datetime(df_daily['dteday'])
if 'dteday' in df_hour.columns:
    df_hour['dteday'] = pd.to_datetime(df_hour['dteday'])

df_daily.head(3)

## Adatáttekintés (Data Understanding)

Röviden áttekintem a főbb oszlopokat és a hiányzó értékeket. A célváltozó `cnt`.
A `casual` és `registered` oszlopokat nem fogom közvetlenül használni a modellezésben (adat‑szivárgás miatt).


In [ ]:
display(df_daily.info())
display(df_daily.describe(include='all').T.head(12))

## EDA – Napi adatok

**Mit figyelek itt?** Szezonális mintázat, időjárási hatások, naptári hatások, korrelációk.
Az alábbi megállapításokat röviden szövegesen is összefoglalom a plotok után.


### Megfigyelések (összefoglaló)

- A bérlések száma erőteljesen szezonális: tavasszal és nyáron emelkedik, télen visszaesik.
- A hőmérséklet pozitívan, míg a páratartalom és a szélsebesség negatívan hat a bérlésszámra.
- Ünnepnapokon általában alacsonyabb a forgalom, munkanapokon magasabb.
- A `registered` felhasználók száma összességében magasabb, de nyáron a `casual` arány is nő.
- A `casual` és `registered` erősen korrelál a `cnt`‑vel, ezért modellezéshez nem használom (leakage).

In [ ]:
# Példa: egyszerű napi idősor
ax = df_daily.set_index('dteday')['cnt'].plot(title='Napi bérlések időben')
ax.set_xlabel('Dátum'); ax.set_ylabel('Bérlések száma')
plt.show()

# Korrelációs mátrix (főbb numerikus oszlopok)
num_cols = ['temp','atemp','hum','windspeed','cnt']
sns.heatmap(df_daily[num_cols].corr(), annot=True, fmt='.2f')
plt.title('Korrelációk (napi adatok)')
plt.show()

## EDA – Óránkénti adatok (Intraday)

**Mit figyelek itt?** Napi ciklus (reggeli és délutáni csúcs), hétvége vs. munkanap, hónapok szerinti mintázat.


### Megfigyelések (összefoglaló)

- Két egyértelmű napi csúcs: reggel és késő délután (ingázás).
- Munkanapokon nagyobb a használat, hétvégén inkább délután, szabadidős jelleggel.
- A havi átlag június–szeptember között tetőzik, január–februárban alacsony.
- A hőmérséklet emelkedésével bizonyos tartományig nő a kereslet, magas páratartalomnál csökken.

In [ ]:
# Óránkénti átlag grafikon (illusztráció)
hr_mean = df_hour.groupby('hr')['cnt'].mean()
hr_mean.plot(kind='line', title='Átlagos bérlésszám óránként')
plt.xlabel('Óra'); plt.ylabel('Átlagos bérlések')
plt.show()

## Feature Engineering

Célom, hogy a szezonális és időjárási hatásokat a modell jól meg tudja tanulni. A `casual` és `registered` oszlopokat nem használom, mert a cél részei (leakage).

**Ciklikus kódolás (napi):**  
`weekday` → `sin_wd`, `cos_wd` ; `mnth` → `sin_mnth`, `cos_mnth`  
(Ezzel a 0. és a maximum érték „szomszédos”, a modell folyamatos mintázatot lát.)

**Megmaradó fő változók:** `temp`, `atemp`, `hum`, `windspeed`, kategóriák: `season`, `yr`, `holiday`, `workingday`, `weathersit`.

**Refinement (SHAP alapján):** a gyenge vagy redundáns jellemzőket elhagytam, a legerősebbeket megtartottam.


In [ ]:
# Ciklikus kódolás (napi adatokhoz)
def fe_daily(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    # sin/cos (hét napja és hónap)
    df['sin_wd'] = np.sin(2*np.pi*df['weekday']/7)
    df['cos_wd'] = np.cos(2*np.pi*df['weekday']/7)
    df['sin_mnth'] = np.sin(2*np.pi*df['mnth']/12)
    df['cos_mnth'] = np.cos(2*np.pi*df['mnth']/12)
    return df

## Baseline pipeline (napi)

Itt egy egyszerű baseline modellt állítok össze `Pipeline`-ban. A kategóriákat OHE-olom, a többi változó passthrough. A célváltozó: `cnt`.


In [ ]:
from sklearn import set_config
set_config(transform_output='pandas')

TARGET = 'cnt'
DROP = ['casual','registered','dteday', TARGET]

cat_cols = ['season','yr','holiday','workingday','weathersit']

pre = ColumnTransformer(
    transformers=[('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols)],
    remainder='passthrough'
)

pipe_baseline = Pipeline(steps=[
    ('fe', FunctionTransformer(fe_daily, validate=False)),
    ('prep', pre),
    ('model', HistGradientBoostingRegressor(random_state=42))
])

# Idő alapú split (egyszerű példa)
split_day = pd.Timestamp('2012-10-01')
train = df_daily[df_daily['dteday'] < split_day].copy()
test  = df_daily[df_daily['dteday'] >= split_day].copy()

X_train = train.drop(columns=DROP).copy()
y_train = train[TARGET].copy()
X_test  = test.drop(columns=DROP).copy()
y_test  = test[TARGET].copy()

pipe_baseline.fit(X_train, y_train)
y_pred_base = pipe_baseline.predict(X_test)

rmse_b = mean_squared_error(y_test, y_pred_base, squared=False)
mae_b  = mean_absolute_error(y_test, y_pred_base)
r2_b   = r2_score(y_test, y_pred_base)

print(f'Baseline RMSE: {rmse_b:.2f} | MAE: {mae_b:.2f} | R²: {r2_b:.3f}')

In [ ]:
# Diagnosztikai ábrák (Seaborn)
res_base = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred_base})
res_base['Residual'] = res_base['Actual'] - res_base['Predicted']

fig, ax = plt.subplots(1,3, figsize=(18,5))

# 1) Predicted vs Actual
sns.scatterplot(data=res_base, x='Actual', y='Predicted', alpha=0.6, ax=ax[0])
sns.lineplot(x=[res_base.min().min(), res_base.max().max()],
             y=[res_base.min().min(), res_base.max().max()],
             color='red', linestyle='--', ax=ax[0])
ax[0].set_title('Predicted vs Actual (baseline)')

# 2) Residuals vs Predicted
sns.scatterplot(data=res_base, x='Predicted', y='Residual', alpha=0.6, ax=ax[1])
sns.lineplot(x=[res_base['Predicted'].min(), res_base['Predicted'].max()],
             y=[0,0], color='red', linestyle='--', ax=ax[1])
ax[1].set_title('Residuals vs Predicted (baseline)')

# 3) Idősoros összevetés
sns.lineplot(x=range(len(y_test)), y=y_test, label='Actual', ax=ax[2])
sns.lineplot(x=range(len(y_pred_base)), y=y_pred_base, label='Predicted', ax=ax[2])
ax[2].set_title('Actual vs Predicted over time (baseline)')
ax[2].legend()

plt.tight_layout(); plt.show()

### Baseline – rövid értékelés

A predikciók nagyjából követik a valós értékeket, de a magasabb csúcsoknál enyhe alulbecslés látszik.
A reziduumok többnyire a nullavonal körül szóródnak, torzítást nem látok. Ez jó kiindulópont a finomításhoz.


## 2. Finomított Pipeline a SHAP-eredmények és a metrikák alapján

Ebben a pipeline-verzióban a SHAP-elemzés és a korábbi modell teljesítménymutatói alapján módosítottam a jellemzők körét és a modell paramétereit.
A cél az volt, hogy csökkentsem az overfittinget, növeljem az általánosítási képességet, és csak a legerősebb jellemzőket tartsam meg.

**Főbb változtatások:**
- Gyenge vagy redundáns jellemzők elhagyása (`temp_hum`, `is_weekend`, `comfort_diff`).
- Megtartott fő prediktorok: `temp`, `atemp`, `hum`, `windspeed` + ciklikus kódok (`sin_mnth`, `cos_mnth`, `sin_wd`, `cos_wd`).
- Enyhe regularizáció és óvatosabb beállítások a HistGBR-ben (pl. kisebb fa-mélység, tanulási ráta finomítása).


In [ ]:
# (Példa) Refined pipeline – a FE ugyanaz a napi sin/cos, de a modell paraméterein szigorítunk
pipe_refined = Pipeline(steps=[
    ('fe', FunctionTransformer(fe_daily, validate=False)),
    ('prep', pre),
    ('model', HistGradientBoostingRegressor(
        random_state=42,
        max_depth=5,
        learning_rate=0.05,
        min_samples_leaf=30,
        max_leaf_nodes=40,
        l2_regularization=0.5
    ))
])

pipe_refined.fit(X_train, y_train)
y_pred_ref = pipe_refined.predict(X_test)

rmse_r = mean_squared_error(y_test, y_pred_ref, squared=False)
mae_r  = mean_absolute_error(y_test, y_pred_ref)
r2_r   = r2_score(y_test, y_pred_ref)

print(f'Refined RMSE: {rmse_r:.2f} | MAE: {mae_r:.2f} | R²: {r2_r:.3f}')

In [ ]:
# Plotok a refined modellhez
res_ref = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred_ref})
res_ref['Residual'] = res_ref['Actual'] - res_ref['Predicted']

fig, ax = plt.subplots(1,3, figsize=(18,5))

sns.scatterplot(data=res_ref, x='Actual', y='Predicted', alpha=0.6, ax=ax[0])
sns.lineplot(x=[res_ref.min().min(), res_ref.max().max()],
             y=[res_ref.min().min(), res_ref.max().max()],
             color='red', linestyle='--', ax=ax[0])
ax[0].set_title('Predicted vs Actual (refined)')

sns.scatterplot(data=res_ref, x='Predicted', y='Residual', alpha=0.6, ax=ax[1])
sns.lineplot(x=[res_ref['Predicted'].min(), res_ref['Predicted'].max()],
             y=[0,0], color='red', linestyle='--', ax=ax[1])
ax[1].set_title('Residuals vs Predicted (refined)')

sns.lineplot(x=range(len(y_test)), y=y_test, label='Actual', ax=ax[2])
sns.lineplot(x=range(len(y_pred_ref)), y=y_pred_ref, label='Predicted', ax=ax[2])
ax[2].set_title('Actual vs Predicted over time (refined)')
ax[2].legend()

plt.tight_layout(); plt.show()

### Refined – rövid értékelés

A validációs hibák (RMSE, MAE) csökkentek a baseline-hoz képest, és az időbeli predikciók stabilabbak.
A csúcsoknál még mindig lehet némi alulbecslés, de a reziduumok „fehérebbek”, kevésbé szerkezettek.


## Időalapú validáció (TimeSeriesSplit)

Az idő szerinti felosztás célja, hogy mindig „múltból jósoljunk jövőre”. Itt átlagos RMSE‑t számolok több foldon.


In [ ]:
tscv = TimeSeriesSplit(n_splits=5)
scores = cross_val_score(
    pipe_refined, X_train, y_train,
    cv=tscv, scoring='neg_root_mean_squared_error'
)
print(f'CV RMSE (átlag): {-scores.mean():.2f} ± {scores.std():.2f}')

## Model Evaluation Summary

A finomított pipeline kiegyensúlyozottabb teljesítményt adott, mint a baseline: a hibák csökkentek, és a predikciók stabilabbak lettek az időbeli foldokban.
A modell erőssége, hogy jól kezeli a szezonális és időjárási hatásokat; ugyanakkor télen és extrém körülmények között előfordulhat nagyobb hiba.
A SHAP‑alapú döntések miatt a modell értelmezhető maradt: a hőmérséklet a legerősebb pozitív tényező, míg a páratartalom és a szél csökkentő hatású.


## Forrás

Bansal, A., Balaji, K., & Lalani, Z. (2025). *Temporal Encoding Strategies for Energy Time Series Prediction.*  
arXiv preprint arXiv:2503.15456. Elérhető: https://arxiv.org/pdf/2503.15456
